# Simple Gmail Agent with Langchain

`requirements.txt`:

```
google-api-python-client
google-auth-oauthlib
google-auth-httplib2
beautifulsoup4
langchain-community
langchain-google-community[gmail]
langchain-google-genai
pillow
```

This notebook demonstrates how to use Langchain to build a simple Gmail agent that can read emails and extract information from them. The agent will be able to extract the sender, subject, and body of the email, and then use Langchain to extract the language of the email and perform named entity recognition on the body of the email.

## Import dependencies

In [1]:
from dotenv import load_dotenv

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

from langchain_openai import ChatOpenAI


## Load the environment variables

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(temperature=0)

## Set up the Gmail API

### Import API

In [4]:
from langchain_google_community import GmailToolkit

toolkit = GmailToolkit()

### Set up the Gmail API

In [5]:
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

### The tools

In [6]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x00000238C336DB50>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x00000238C336DB50>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x00000238C336DB50>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x00000238C336DB50>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x00000238C336DB50>)]

In [7]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)


In [8]:
agent = create_openai_tools_agent(llm, toolkit.get_tools(), prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit.get_tools(),
    verbose=True, #set it True to see what happens behind the scenes
)


In [11]:
agent_executor.invoke(
    {"input": "Could you check at what time I was last emailed?"}
)




> Entering new AgentExecutor chain...

Invoking: `search_gmail` with `{'query': 'to:me', 'max_results': 1}`


[{'id': '1913bad96874560b', 'threadId': '1913bad96874560b', 'snippet': 'Rana Adeel, respond to Muneeb&#39;s invite now. ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏', 'body': 'Rana Adeel Tahir, respond to Muneeb’s invite now.\r\n        \r\nHi Rana Adeel, I’d like to join your professional network\r\n\r\nMuneeb Raza Toor\r\nStudent at National University of Computer and Emerging Sciences\r\nFaisalabad\r\n29 connections in common\r\nSee all connections in common:https://www.linkedin.com/comm/search/results/people/?facetNetwork=%5B%22F%22%5D&facetConnectionOf=%5B%22ADoAAFDtSdMB6FsDdzRxsYRYi2v4F_AcFXHjC2U%22%5D&origin=SHARED_CONNECTIONS_CANNED_SEARCH&lipi=urn%3Ali%3Apage%3Aemail_email_m2m_invite_single_01%3BnDqSZgioTE23TPZJPc%2FzTw%3D%3D&midToken=AQHIuKVFxcePpA&midSig=1KM

{'input': 'Could you check at what time I was last emailed?',
 'output': 'The last email you received was from Muneeb Raza Toor with the subject "I want to connect." It was received recently.'}